In [2]:
import cv2
start_h, end_h, start_w, end_w = 0, 0, 0, 0 # The height and width of the character area start and end 
image = cv2.imread('C:/Users/ratne/Downloads/demo.jpg', 0) # Read directly as a grayscale image 
cv2.imshow('img_GRAY', image)
_, image = cv2.threshold(image, 50, 255, cv2.THRESH_BINARY) # Two valued 
cv2.imshow('img_BINARY', image)
# Get rid of the noise 
kernel1 = cv2.getStructuringElement(cv2.MORPH_RECT, (7, 7)) # Simple corrosion expansion core 
kernel2 = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5)) # Simple corrosion expansion core 
image = cv2.erode(image, kernel=kernel1) # corrosion 
image = cv2.dilate(image, kernel=kernel2) # inflation 
cv2.imshow('img_denoise', image)
h, w = image.shape # The height and width of the original 
# print(h, w)"C:\Users\ratne\Downloads\allloha.jpg"
list1 = [] # Column sum 
list2 = [] # Row sum 
for i in range(w):
    list1.append(1 if image[:, i].sum() != 0 else 0) # Summation , Not for 0 Set up 1
    for i in range(h):
        list2.append(1 if image[i, :].sum() != 0 else 0) # To sum up , Not for 0 Set up 1
# print(len(list1))
# print(len(list2))
# Crop character region 
# Find the range of lines 
flag = 0
for i, e in enumerate(list1):
    if e != 0:
        if flag == 0:
            # The first one is not for 0 Location record of 
            start_w = i
            flag = 1
        else:
            # The last one is not 0 The location of 
            end_w = i
            # Find the range of columns 
            flag = 0
for i, e in enumerate(list2):
    if e != 0:
        if flag == 0:
            # The first one is not for 0 Location record of 
            start_h = i
            flag = 1
        else:
            # The last one is not 0 The location of 
            end_h = i
print(start_w, end_w) # Row index range 
print(start_h, end_h) # Column index range 
cv2.imshow('img_number', image[start_h:end_h, start_w:end_w])
l = ([i for i, e in enumerate(list1) if e != 0]) # Columns and lists are not 0 The index of 
# print(l)
img_list = [] # Split digital picture storage list 
temp = [] # Store all row index values of a number 
n = 0 # Number of digital pictures 
for x in l:
    temp.append(x)
    if x+1 not in l:
        # Index discontinuities 
        if len(temp) != 1:
            start_w = min(temp) # Index min 
            end_w = max(temp) # Index max 
            img_list.append(image[start_h:end_h, start_w:end_w]) # The index includes a number slice 
            n += 1
# print(temp)
temp = []
print(n) # Number of characters 
for i in range(n):
    # Display saved characters 
    cv2.imshow('number'+str(i), img_list[i])
    cv2.imwrite('C:/Users/ratne/Downloads//demo'+str(i+1).zfill(2)+'.jpg', img_list[i])
cv2.waitKey(0)

362 361
0 55761


error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [1]:
import cv2
#image = cv2.imread('C:/Users/ratne/Downloads/allloha.jpg') # Read directly as a grayscale image 
#cv2.imshow('img_GRAY', image)
image = cv2.imread('C:/Users/ratne/Downloads/demo.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2.imshow('hey',gray)


In [1]:
# Import the necessary packages
import argparse
import cv2
import numpy as np
from keras.models import load_model
from tensorflow.keras.utils import img_to_array
import functools
 
# Construct the argument parser and parse the arguments
#ap = argparse.ArgumentParser()
#ap.add_argument("-i", "C:/Users/ratne/Downloads/demo.jpg", required=True, help="Path to the image")
#ap.add_argument("-m", "C:/Users/ratne/Downloads/model_hand.h5", required=True, help="Path to the pre-trained model")
#args = vars(ap.parse_args())

###############################################
# This takes two stages
# The first stage is to segment characters
# The second stage is to recognise characters
###############################################

###############################################
# The first stage
###############################################

# Read the image and convert to grayscale
image = cv2.imread('C:/Users/ratne/Downloads/demo.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Show the original image
cv2.imshow("License Plate", image)

# Apply Gaussian blurring and thresholding 
# to reveal the characters on the license plate
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
thresh = cv2.adaptiveThreshold(blurred, 255,
	cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 45, 15)

# Perform connected components analysis on the thresholded images and
# initialize the mask to hold only the components we are interested in
_, labels = cv2.connectedComponents(thresh)
mask = np.zeros(thresh.shape, dtype="uint8")

# Set lower bound and upper bound criteria for characters
total_pixels = image.shape[0] * image.shape[1]
lower = total_pixels // 70 # heuristic param, can be fine tuned if necessary
upper = total_pixels // 20 # heuristic param, can be fine tuned if necessary

# Loop over the unique components
for (i, label) in enumerate(np.unique(labels)):
	# If this is the background label, ignore it
	if label == 0:
		continue
 
	# Otherwise, construct the label mask to display only connected component
	# for the current label
	labelMask = np.zeros(thresh.shape, dtype="uint8")
	labelMask[labels == label] = 255
	numPixels = cv2.countNonZero(labelMask)
 
	# If the number of pixels in the component is between lower bound and upper bound, 
	# add it to our mask
	if numPixels > lower and numPixels < upper:
		mask = cv2.add(mask, labelMask)

# Find contours and get bounding box for each contour
cnts, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
boundingBoxes = [cv2.boundingRect(c) for c in cnts]

# Sort the bounding boxes from left to right, top to bottom
# sort by Y first, and then sort by X if Ys are similar
def compare(rect1, rect2):
    if abs(rect1[1] - rect2[1]) > 10:
        return rect1[1] - rect2[1]
    else:
        return rect1[0] - rect2[0]
boundingBoxes = sorted(boundingBoxes, key=functools.cmp_to_key(compare) )


cv2.rectangle(image, (x,y), (x+w,y+h), (0, 255, 0), 2)

cv2.imshow('hello',image)


NameError: name 'x' is not defined